In [122]:
import os

athena_db = 'trivago'
s3_bucket = os.environ['BUCKET']
date = '2019-08-01'

In [123]:
import awswrangler
session = awswrangler.Session()

In [124]:
import pandas

# Get USPs per accommodation and add destination_id

In [125]:
def get_destination_ids(accommodation_ids):
    with open('query.sql') as f:
        df = session.pandas.read_sql_athena(
            sql=f.read().format(date=repr(date)),
            database=athena_db
        )
        
    return df[df.accommodation_id.isin(accommodation_ids)]

In [126]:
def get_filtered_usps():
    df = session.pandas.read_sql_athena(
        sql="SELECT * FROM unique_selling_points WHERE date = {date}".format(date=repr(date)),
        database=athena_db
    )
    
    return df

In [127]:
usp_df = get_filtered_usps()
destination_ids = get_destination_ids(usp_df["accommodation_id"].values)
usp_df = usp_df.merge(destination_ids, how="left", on="accommodation_id")

In [128]:
usp_df.sample(5)

,accommodation_id,accommodation_ns,usp,date,destination_id
66660,5812,100,Business hotel,2019-08-01,8181
99415,199414,100,Pet friendly hotel,2019-08-01,NaN
99961,161163,100,Breakfast,2019-08-01,NaN
169679,62828,100,Food & drinks,2019-08-01,NaN
253851,148466,100,Pet friendly hotel,2019-08-01,NaN


In [129]:
usp_df.groupby("usp")['destination_id'].nunique().sort_values(ascending=False)

usp
Business hotel        2410
Sleeping comfort      2323
Pet friendly hotel    2291
sea_view              2273
Room quality          1870
                      ... 
sunday_brunch           10
live_music               9
skyline_view             8
japanese_style           7
blackout_curtains        3
Name: destination_id, Length: 63, dtype: int64

In [130]:
session.pandas.to_parquet(
    dataframe=usp_df,
    path=f"s3://{s3_bucket}/usp/output",
    partition_cols=["date"],
)

['s3://thisara-test-athenabucket/usp/output/date=2019-08-01/6f55dc8b1fed437cafbf83a6ad7a54a1.parquet',
 's3://thisara-test-athenabucket/usp/output/date=2019-08-01/6334bec9765948f0b99e38c9dadf7b01.parquet']